In [1]:
import cv2
import numpy as np
import os

In [2]:
data_folder_path = r'D:\UNI\Year 3 term 2\computer vision\computer vision final project\Celebrity Faces Dataset' 

In [3]:
test_data = r'D:\UNI\Year 3 term 2\computer vision\computer vision final project\test data' 

In [4]:
def detect_face(img):
    if len(img.shape) == 3:  # Check if input image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif len(img.shape) == 2:  # If input image has 2 channels (already grayscale)
        gray = img
    else:
        print("Invalid input image format")
        return None, None
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    if len(faces) == 0:
        return None, None
    (x, y, w, h) = faces[0]
    return gray[y:y+w, x:x+h], faces[0]


In [5]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    label_map = {}  # Dictionary to map names to integers
    current_label = 1

    for dir_name in dirs:
        if not dir_name.startswith("."):
            subject_dir_path = data_folder_path + "/" + dir_name
            subject_images_names = os.listdir(subject_dir_path)
            for image_name in subject_images_names:
                if image_name.startswith("."):
                    continue
                image_path = subject_dir_path + "/" + image_name
                image = cv2.imread(image_path)
                face, rect = detect_face(image)
                if face is not None:
                    # Check if the name is already in the label map
                    if dir_name not in label_map:
                        label_map[dir_name] = current_label
                        current_label += 1
                    faces.append(face)
                    labels.append(label_map[dir_name])
    return faces, labels, label_map  # Return the mapping as well

# When calling the function, also receive the label map
faces, labels, label_map = prepare_training_data(data_folder_path)

In [6]:
# Convert lists to numpy arrays
labels = np.array(labels)

In [7]:
# Initialize the LBPH face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
# Train the recognizer on the training images
face_recognizer.train(faces, labels)
# Save the trained model
face_recognizer.save('face_recognition_model.yml')

In [8]:
# Function to draw rectangle on image
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [9]:
# Function to draw text on image
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [10]:
def prepare_testing_data(test_data):
    dirs = os.listdir(test_data)
    test_faces = []
    test_labels = []
    test_label_map = {}  # Dictionary to map names to integers
    current_label = 1

    for dir_name in dirs:
        if not dir_name.startswith("."):
            subject_dir_path = test_data + "/" + dir_name
            subject_images_names = os.listdir(subject_dir_path)
            for image_name in subject_images_names:
                if image_name.startswith("."):
                    continue
                image_path = subject_dir_path + "/" + image_name
                image = cv2.imread(image_path)
                face, rect = detect_face(image)
                if face is not None:
                    # Check if the name is already in the label map
                    if dir_name not in test_label_map:
                        test_label_map[dir_name] = current_label
                        current_label += 1
                    test_faces.append(face)
                    test_labels.append(test_label_map[dir_name])
    return test_faces, test_labels, test_label_map  # Return the mapping as well

# When calling the function, also receive the label map
test_faces, test_labels, test_label_map = prepare_training_data(test_data)

In [11]:
def predict(test_img):
    # Assuming you have loaded your face recognition model as 'face_recognizer'
    # and defined 'label_map' which maps names to integers
    
    # Predict the label for the test image
    label = face_recognizer.predict(test_img)
    
    # Convert predicted label to name
    predicted_name = None
    for name, idx in label_map.items():
        if idx == label:
            predicted_name = name
            break
    
    return predicted_name

In [12]:
def test_recognizer(test_faces, test_labels, test_label_map):
    correct_predictions = 0
    total_predictions = len(test_faces)

    for i in range(total_predictions):
        test_img = test_faces[i]
        true_label = test_labels[i]

        # Predict the label for the test image
        predicted_label = face_recognizer.predict(test_img)

        # Check if the predicted label matches the true label
        if predicted_label[0] == true_label:  # Compare individual elements
            correct_predictions += 1

        print("True Label:", true_label, "Predicted Label:", predicted_label[0])

    accuracy = (correct_predictions / total_predictions) * 100
    print("Accuracy:", accuracy, "%")


In [13]:
#call the recognier function 
test_recognizer(test_faces,test_labels,label_map)

True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 1 Predicted Label: 1
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 2 Predicted Label: 2
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 3 Predicted Label: 3
True Label: 4 Predicted Label: 4
True Label: 4 Predicted Label: 4
True Label

In [17]:
# Load the trained face recognition model
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('face_recognition_model.yml')

# Initialize the camera capture
video_capture = cv2.VideoCapture(0)

# Define a function to detect faces in an image
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    return faces, gray

# Define a function to recognize and annotate faces in an image
def recognize_faces(image):
    faces, gray = detect_faces(image)
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        label, confidence = face_recognizer.predict(face)
        #lower values for more strict recognition.
        if confidence < 100:
            # Get the name corresponding to the predicted label
            label_text = str(label)
        else:
            label_text = "Unknown"
        cv2.rectangle(image,
         (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, label_text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
    return image
# Start the video capture loop and display the recognized faces in real-time
while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    frame = recognize_faces(frame)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close any open windows
video_capture.release()
cv2.destroyAllWindows()